In [5]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [6]:
LOCAL_PATH = 'data'
RAW_TRACE = 'bilibili.pcapng'
TCP_TRACE_FEATURE_FILE = 'bilibili_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'bilibili_udp_pkt.csv'
TCP_FLOW_FEATURE_FILE = 'bilibili_tcp_flow.csv'
UDP_FLOW_FEATURE_FILE = 'bilibili_udp_flow.csv'
BUCKET_NAME = 'edu.nyu.hsn.ddos-data' # replace with your bucket name
KEY = 'CAP_NIC1_00931_20130727230801.dms' # replace with your object key

In [7]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [14]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) ,os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Conversion done
CPU times: user 17.1 ms, sys: 30.4 ms, total: 47.5 ms
Wall time: 8.08 s
Conversion done
CPU times: user 2.44 s, sys: 181 ms, total: 2.62 s
Wall time: 11.9 s


In [15]:
def to_csv(trace_df, extract_func, file):
    extract_func(trace_df, 1.0, upsampled=True).to_csv(file, index=False)

In [16]:
from python import flow_feature
reload(flow_feature)

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE))
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE))
    else:
        raise

UDP 1.0 samp rate with upsampling: 100%|██████████| 14/14 [00:00<00:00, 131.97it/s]
